In [32]:
import numpy as np
import geopandas as gpd
import shapely.geometry as sgeo

This notebooks creates a hexagon grid over the study area. Inspired by [this implementation](https://sabrinadchan.github.io/data-blog/building-a-hexagonal-cartogram.html).

In [33]:
if "snakemake" in locals():
    input_path = snakemake.input[0]
    output_path = snakemake.output[0]
    projection = snakemake.params["projection"]
    hexagon_size = snakemake.params["hexagon_size"]
else:
    input_path = "../../resources/copenhagen.geojson"
    output_path = "../../results/hexagons.gpkg"
    projection = "EPSG:25832"
    hexagon_size = 500

In [34]:
# Load area
df_area = gpd.read_file(input_path).to_crs(projection)

In [35]:
# Construct hexagons
xmin, ymin, xmax, ymax = df_area.total_bounds

a = np.sin(np.pi / 3)
cols = np.arange(np.floor(xmin), np.ceil(xmax), 3 * hexagon_size)
rows = np.arange(np.floor(ymin) / a, np.ceil(ymax) / a, hexagon_size)

hexagons = []
for x in cols:
    for i, y in enumerate(rows):
        if i % 2 == 0:
            x0 = x
        else:
            x0 = x + 1.5 * hexagon_size

        hexagons.append(sgeo.Polygon([
            (x0, y * a),
            (x0 + hexagon_size, y * a),
            (x0 + (1.5 * hexagon_size), (y + hexagon_size) * a),
            (x0 + hexagon_size, (y + (2 * hexagon_size)) * a),
            (x0, (y + (2 * hexagon_size)) * a),
            (x0 - (0.5 * hexagon_size), (y + hexagon_size) * a),
        ]))

df_grid = gpd.GeoDataFrame({'geometry': hexagons}, crs = projection)
df_grid = gpd.sjoin(df_grid, df_area, predicate = "within")[["geometry"]]
df_grid["grid_id"] = np.arange(len(df_grid))

In [36]:
df_grid.to_file(output_path)